In [52]:
import pandas as pd
import numpy as np
from datetime import datetime

In [78]:


Gas_prices=pd.read_csv("FINAL_spark_spreads").iloc[:,1:]
Oil_prices=pd.read_csv("Oil_prices.csv").iloc[:,1:]
Oil_prices['Oil_Price'] = Oil_prices['Oil_Price'].str.replace(',', '.').apply(pd.to_numeric)
Oil_prices['Oil_Volume'] = Oil_prices['Oil_Volume'].str.replace(' ', '').apply(pd.to_numeric)
Sun_energy=pd.read_csv("Sun_energy.csv").iloc[:,1:]
Wind_energy=pd.read_csv("Wind_energy (2).csv").iloc[:,1:]
Coal_prices=pd.read_csv("Coal_future_price.csv").iloc[:,1:]
Industrial_prod=pd.read_csv("Industrial_prod.csv").iloc[:,1:]
print(Industrial_prod)


Energy_prod_df=pd.read_csv("Energy_production.csv").iloc[:,1:]
shipping_index=pd.read_csv("Shipping_prices.csv").iloc[:,1:]
shipping_index['Shipping_price'] = shipping_index['Shipping_price'].str.replace(' ', '')
shipping_index['Gas_consumtion'] = shipping_index['Gas_consumtion'].str.replace(' ', '')
shipping_index['Shipping_price'] = shipping_index['Shipping_price'].str.replace(',', '.').astype(float)
shipping_index['Gas_consumtion'] = shipping_index['Gas_consumtion'].str.replace(',', '.').astype(float)





Google_data=pd.read_csv("Google_data.csv").iloc[:,1:]


Industrial_mat=pd.read_csv("PINDUINDEXM.csv")
Industrial_mat['Date'] = pd.to_datetime(Industrial_mat['DATE'])
daily_dates = pd.date_range(start=Industrial_mat['Date'].min(), end=Gas_prices['Date'].max(), freq='D')
new_df = pd.DataFrame({'Date': daily_dates})
Ind_df = new_df.merge(Industrial_mat, on='Date', how='outer')
Ind_df['PINDUINDEXM'].fillna(method='ffill', inplace=True)
Ind_df = Ind_df.drop('DATE', axis=1)
print(Ind_df)

Temperatures=pd.read_csv("Mean_Temperatures.csv").iloc[:,1:]

Storage=pd.read_csv("StorageData.csv", delimiter=';').iloc[:,1:]
Big_comps=pd.read_csv("Big_energy_companies.csv").iloc[:,1:]

Big_comps.columns = ['Date', 'EOAN', "XOM","ONGC","EQNR"]


Storage["Net_Injection"]=Storage["Injection (GWh/d)"]-Storage["Withdrawal (GWh/d)"]
Storage["Date"]=Storage["Gas Day Start"].values
Storage=Storage.iloc[:,[-1,2,-2]]


Merged_df=pd.merge(Gas_prices,Oil_prices, on="Date")
Merged_df=pd.merge(Merged_df,Sun_energy, on="Date")
Merged_df=pd.merge(Merged_df,Wind_energy, on="Date")
Merged_df=pd.merge(Merged_df,Temperatures, on="Date")
Merged_df=pd.merge(Merged_df,Storage, on="Date")
Merged_df=pd.merge(Merged_df,shipping_index, on="Date")
Merged_df=pd.merge(Merged_df,Google_data, on="Date")
Merged_df=pd.merge(Merged_df,Industrial_prod, on="Date")
Ind_df['Date'] = Ind_df['Date'].astype(str)
Merged_df=pd.merge(Merged_df,Ind_df, on="Date")
Merged_df["Target"]= np.log(Merged_df['Gas_Price'] / Merged_df['Gas_Price'].shift(1))


#Lags

lags = [30,60,100] # Add lag variables for the previous 3 time steps

# Add lag variables to the DataFrame
for column in ['EU_clean_spark_spread',"Average_temp","Net_Injection","Oil_Price","Wind_energy","Electricity_price","PINDUINDEXM","Industrial_prod",  "Google_data","Shipping_price","Gas_consumtion","Target"]:
    for lag in lags:
        Merged_df[f'{column}_EMA{lag}'] = Merged_df[column]/Merged_df[column].rolling(window=lag).mean()
    Merged_df[f'{column}_EMA_mean'] = Merged_df[column].rolling(window=7).mean()/Merged_df[column].rolling(window=265).mean()
Merged_df["R2"]=Merged_df["Target"]*Merged_df["Target"]
Merged_df["Relav_R2"]=Merged_df["R2"].rolling(window=7).mean()/Merged_df["R2"].rolling(window=365).mean()
oil_return=np.log(Merged_df['Oil_Price'] / Merged_df['Oil_Price'].shift(1))
Merged_df["Oil_R2"]=(oil_return*oil_return).rolling(window=7).mean()/(oil_return*oil_return).rolling(window=365).mean()

Merged_df=Merged_df.dropna()

#Energy_production and BIg Coal

Merged_df=pd.merge(Merged_df,Coal_prices, on="Date")
Merged_df=pd.merge(Merged_df,Energy_prod_df, on="Date")



#Transform Temperature data
def transform_function(x):
    return max(x-20,0)# Example transformation: doubling the value

def transform_function_2(x):
    return max(16.5-x,0)# Example transformation: doubling the value

# Create the new column using the assign() function
Merged_df = Merged_df.assign(Cooling_req=Merged_df['Average_temp'].apply(transform_function))
Merged_df = Merged_df.assign(Heating_req=Merged_df['Average_temp'].apply(transform_function_2))




Merged_df=pd.merge(Merged_df,Big_comps, on="Date")
column = Merged_df.pop("Gas_Price")
Merged_df["Gas_Price"] = column
Merged_df=Merged_df.dropna()

print(Merged_df)
print(Merged_df["Target"])
print

Merged_df.to_csv("Final_Dataset.csv")








            Date  Industrial_prod
0     2007-05-01         97.56831
1     2007-05-02         97.56831
2     2007-05-03         97.56831
3     2007-05-04         97.56831
4     2007-05-05         97.56831
...          ...              ...
5931  2023-07-27         94.56004
5932  2023-07-28         94.56004
5933  2023-07-29         94.56004
5934  2023-07-30         94.56004
5935  2023-07-31         94.56004

[5936 rows x 2 columns]
           Date  PINDUINDEXM
0    2010-01-01   155.128898
1    2010-01-02   155.128898
2    2010-01-03   155.128898
3    2010-01-04   155.128898
4    2010-01-05   155.128898
...         ...          ...
4994 2023-09-04   154.559147
4995 2023-09-05   154.559147
4996 2023-09-06   154.559147
4997 2023-09-07   154.559147
4998 2023-09-08   154.559147

[4999 rows x 2 columns]
            Date  Electricity_price  Exchange  CO2_spot  EU_spark_spread  \
0     2012-01-03          35.026250   1.30505     7.640       -28.153070   
1     2012-01-04          32.163333   1.29